# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

#### Import the mnist dataset from keras datasets

In [1]:
import numpy as np

from keras.datasets import mnist

Using TensorFlow backend.


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
#datasets below 5 digits
x_train_b5 = x_train[y_train < 5]
y_train_b5 = y_train[y_train < 5]
x_test_b5 = x_test[y_test < 5]
y_test_b5 = y_test[y_test < 5]

#datasets above 5 digits
x_train_a5 = x_train[y_train >= 5]
y_train_a5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_a5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_a5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [4]:
print(f'Shape of x_train is {x_train.shape}.')
print(f'Shape of y_train is {y_train.shape}.')
print(f'Shape of x_test is {x_test.shape}.')
print(f'Shape of y_test is {y_test.shape}.')

Shape of x_train is (60000, 28, 28).
Shape of y_train is (60000,).
Shape of x_test is (10000, 28, 28).
Shape of y_test is (10000,).


In [5]:
print(x_train_b5.shape)
print(y_train_b5.shape)
print(x_test_b5.shape)
print(y_test_b5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [6]:
print(x_train_a5.shape)
print(y_train_a5.shape)
print(x_test_a5.shape)
print(y_test_a5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [7]:
from keras import backend as K
print(K.image_data_format())

channels_last


In [0]:
x_train_b5 = x_train_b5.reshape((30596, 28, 28, 1))
x_test_b5 = x_test_b5.reshape((5139, 28, 28, 1))

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [0]:
x_train_b5 = x_train_b5.astype('float32')
x_test_b5 = x_test_b5.astype('float32')
x_train_b5 /= 255
x_test_b5 /= 255

### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [10]:
print('X_train shape:', x_train_b5.shape)
print('X_test shape:', x_test_b5.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [0]:
classes = len(set(y_test))

In [0]:
y_train_b5 = to_categorical(y_train_b5, num_classes=classes)
y_test_b5 = to_categorical(y_test_b5, num_classes=classes)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [14]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

W0707 08:34:30.201012 140088000448384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 08:34:30.218308 140088000448384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 08:34:30.221576 140088000448384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0707 08:34:30.242835 140088000448384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0707 08:34:30.246217 140088000448384 deprecation_wrapp

### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [15]:
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

W0707 08:34:30.396915 140088000448384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0707 08:34:30.425610 140088000448384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



### Print the training and test accuracy for 5 epochs

In [16]:
model.fit(x_train_b5, y_train_b5, batch_size=32, epochs=5, validation_data=(x_test_b5, y_test_b5))

W0707 08:34:30.562882 140088000448384 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 9s 304us/step - loss: 0.1534 - acc: 0.9500 - val_loss: 0.0260 - val_acc: 0.9930
Epoch 2/5
30596/30596 [==============================] - 7s 243us/step - loss: 0.0467 - acc: 0.9853 - val_loss: 0.0105 - val_acc: 0.9967
Epoch 3/5
30596/30596 [==============================] - 7s 243us/step - loss: 0.0352 - acc: 0.9891 - val_loss: 0.0101 - val_acc: 0.9961
Epoch 4/5
30596/30596 [==============================] - 7s 244us/step - loss: 0.0271 - acc: 0.9919 - val_loss: 0.0113 - val_acc: 0.9961
Epoch 5/5
30596/30596 [==============================] - 8s 250us/step - loss: 0.0267 - acc: 0.9924 - val_loss: 0.0064 - val_acc: 0.9975


In [17]:
sc = model.evaluate(x_test_b5, y_test_b5, verbose=0)
print(f'Test Score {sc[0]}, Test Accuracy {sc[1]}')

Test Score 0.006447878525380851, Test Accuracy 0.9974703249659467


In [0]:
model.save('model_b5.h5')

### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [0]:
from keras.models import load_model

In [0]:
below4Model = load_model('model_b5.h5')

In [21]:
for layers in below4Model.layers:
    print(below4Model.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + ' is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

sequential_1
conv2d_1 is not trainable

sequential_1
max_pooling2d_1 is not trainable

sequential_1
dropout_1 is not trainable

sequential_1
conv2d_2 is not trainable

sequential_1
max_pooling2d_2 is not trainable

sequential_1
dropout_2 is not trainable

sequential_1
flatten_1 is not trainable

sequential_1
dense_1 is trainable

sequential_1
dropout_3 is not trainable

sequential_1
dense_2 is trainable



### Do the required preprocessing for `x_train_a5` also same as `x_train_b5` and for `y_train_a5` same as `y_train_b5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [22]:
x_train_a5.shape, x_test_a5.shape

((29404, 28, 28), (4861, 28, 28))

In [0]:
x_train_a5 = x_train_a5.reshape((29404, 28, 28, 1))
x_test_a5 = x_test_a5.reshape((4861, 28, 28, 1))
x_train_a5 = x_train_a5.astype('float32')
x_test_a5 = x_test_a5.astype('float32')
x_train_a5 /= 255
x_test_a5 /= 255

y_train_a5 = to_categorical(y_train_a5, num_classes=classes)
y_test_a5 = to_categorical(y_test_a5, num_classes=classes)

### Check

Verify the shapes with the given below.

In [24]:
print(x_train_a5.shape)
print(y_train_a5.shape)
print(x_test_a5.shape)
print(y_test_a5.shape)

(29404, 28, 28, 1)
(29404, 10)
(4861, 28, 28, 1)
(4861, 10)


## Print the accuracy for classification of digits 5 to 9

In [0]:
below4Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [26]:
below4Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
__________

In [27]:
below4Model.fit(x_train_a5, y_train_a5, batch_size=32, epochs=5, validation_data=(x_test_a5, y_test_a5))

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 6s 191us/step - loss: 0.3109 - acc: 0.9101 - val_loss: 0.0515 - val_acc: 0.9844
Epoch 2/5
29404/29404 [==============================] - 5s 177us/step - loss: 0.0952 - acc: 0.9702 - val_loss: 0.0365 - val_acc: 0.9881
Epoch 3/5
29404/29404 [==============================] - 5s 177us/step - loss: 0.0746 - acc: 0.9780 - val_loss: 0.0294 - val_acc: 0.9897
Epoch 4/5
29404/29404 [==============================] - 5s 177us/step - loss: 0.0685 - acc: 0.9777 - val_loss: 0.0277 - val_acc: 0.9916
Epoch 5/5
29404/29404 [==============================] - 5s 177us/step - loss: 0.0569 - acc: 0.9820 - val_loss: 0.0234 - val_acc: 0.9924


In [28]:
sc = below4Model.evaluate(x_test_a5, y_test_a5, verbose=0)
print(f'Test Score {sc[0]}, Test Accuracy {sc[1]}')

Test Score 0.023357077801853095, Test Accuracy 0.9923883974490846


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [31]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

I0707 08:35:40.253712 140088000448384 twenty_newsgroups.py:247] Downloading 20news dataset. This may take a few minutes.
I0707 08:35:40.257681 140088000448384 twenty_newsgroups.py:80] Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [33]:
twenty_test.data[0:2]

["From: brian@ucsd.edu (Brian Kantor)\nSubject: Re: HELP for Kidney Stones ..............\nOrganization: The Avant-Garde of the Now, Ltd.\nLines: 12\nNNTP-Posting-Host: ucsd.edu\n\nAs I recall from my bout with kidney stones, there isn't any\nmedication that can do anything about them except relieve the pain.\n\nEither they pass, or they have to be broken up with sound, or they have\nto be extracted surgically.\n\nWhen I was in, the X-ray tech happened to mention that she'd had kidney\nstones and children, and the childbirth hurt less.\n\nDemerol worked, although I nearly got arrested on my way home when I barfed\nall over the police car parked just outside the ER.\n\t- Brian\n",
 'From: rind@enterprise.bih.harvard.edu (David Rind)\nSubject: Re: Candida(yeast) Bloom, Fact or Fiction\nOrganization: Beth Israel Hospital, Harvard Medical School, Boston Mass., USA\nLines: 37\nNNTP-Posting-Host: enterprise.bih.harvard.edu\n\nIn article <1993Apr26.103242.1@vms.ocom.okstate.edu>\n banschbach@

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [34]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [35]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [36]:
twenty_train.data[0:2]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [0]:
X_train = twenty_train.data
X_test = twenty_test.data
y_train = twenty_train.target
y_test = twenty_test.target

In [0]:
# TfidfVectorizer
vect = TfidfVectorizer()

In [0]:
X_train_tfidf = vect.fit_transform(X_train).todense()
y_train_tfidf = y_train

In [0]:
df = pd.DataFrame(X_train_tfidf, columns=vect.get_feature_names())

In [42]:
df.head()

,00,000,0000,0000001200,000005102000,0001,000100255pixel,00014,000406,0007,000usd,0010,001004,0010580b,001125,001200201pixel,0014,001642,00196,002,0028,003258u19250,0033,0038,0039,004021809,004158,004627,0049,00500,005148,00630,008561,0094,00am,00index,00pm,01,0100,010116,...,zoerasterism,zola,zolf,zolft,zoloft,zombie,zonal,zone,zonker,zoo,zooid,zool,zoom,zooming,zooms,zopfi,zorasterism,zorg,zorn,zrz,zsoft,zt,zubkoff,zues,zug,zumder,zur,zurich,zurlo,zus,zvi,zvonko,zwart,zyeh,zyklon,zyxel,zz,zzz,ªl,íålittin
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.29521,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.055283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057185,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [44]:
logreg = LogisticRegression()
logreg.fit(df, y_train_tfidf)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
X_test_tfidf = vect.transform(X_test)
y_test_tfidf = y_test

In [46]:
y_pred_class = logreg.predict(X_test_tfidf)
y_pred_class

array([2, 2, 2, ..., 2, 2, 1])

In [47]:
print(metrics.accuracy_score(y_test_tfidf, y_pred_class))

0.8868175765645806
